# 03-Feature Engineering

Bu notebook’ta, temizlenmiş eğitim verisi üzerinde model için yeni değişkenler (feature’lar) deniyorum.

**Amaç:**

- Modele ekstra sinyal verecek yeni değişkenler üretmek,
- Delinquency (gecikme) bilgisini daha kompakt ve okunur feature’lara çevirmek,
- Limit ve borç kullanımına dair basit risk göstergeleri türetmek,
- Sağa çarpık dağılıma sahip değişkenler için uygun dönüşümler denemek.

Bu notebook daha çok deneme/analiz içindir; sonunda mantıklı bulduğum adımları `src/data_preprocessing.py` dosyasında fonksiyonlar hâline getiriyorum.


# CHAPTER 1 : Core Transformations

### 1. Kurulum ve Temizlenmiş Verinin Yüklenmesi

Bu adımda:

- Notebook’tan proje kök klasörünü görebilmek için bir üst klasörü (`..`) `sys.path`’e ekliyorum.
- Böylece `src` paketi ve içindeki `config.py` dosyası import edilebilir hâle geliyor.
- `src.config` içindeki `CLEAN_TRAIN` yolunu kullanarak, data cleaning aşamasında oluşturduğum **temiz eğitim verisini** yüklüyorum.
- Verinin yapısını hızlıca görmek için ilk birkaç satırı ekrana bastırıyorum.

In [1]:
import sys
import os

# Notebook'tan bir üst klasörü (proje kökü) Python path'ine ekle
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Config'ten temiz eğitim veri yolunu al
from src.config import CLEAN_TRAIN

# Grafik ayarları
sns.set(style="whitegrid", palette="Set2")
plt.rcParams["figure.figsize"] = (10, 5)

# Temizlenmiş eğitim verisini yükle
df = pd.read_csv(CLEAN_TRAIN)
df.head()


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45.0,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40.0,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38.0,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30.0,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49.0,1,0.024926,63588.0,7,0,1,0,0.0


## 2. Delinquency Özet Değişkenleri

Üç gecikme kolonu kullanıyorum:

- `NumberOfTime30-59DaysPastDueNotWorse`
- `NumberOfTime60-89DaysPastDueNotWorse`
- `NumberOfTimes90DaysLate`

Bu üç kolonu tek tek kullanmak yerine, gecikme bilgisini daha toplu görmek için şu özet değişkenleri üretiyorum:

- **TotalDelinquency**: Üç gecikme sayısının toplamı.
- **EverDelinquent**: Herhangi bir tipte en az bir gecikmesi olan müşteri (0/1).
- **Ever90DaysLate**: 90+ gün gecikme yaşamış müşteri bayrağı (0/1).

Bu özetler sayesinde, hem grafikleri okumak hem de modelde “gecikme var mı / ne kadar ağır?” sorusunu cevaplamak daha kolay oluyor.

In [3]:
delinq_cols = [
    "NumberOfTime30-59DaysPastDueNotWorse",
    "NumberOfTime60-89DaysPastDueNotWorse",
    "NumberOfTimes90DaysLate",
]

# Toplam gecikme sayısı
df["TotalDelinquency"] = df[delinq_cols].sum(axis=1)

# Herhangi bir gecikme yaşanmış mı?
df["EverDelinquent"] = (df["TotalDelinquency"] > 0).astype(int)

# 90+ gün gecikme yaşanmış mı?
df["Ever90DaysLate"] = (df["NumberOfTimes90DaysLate"] > 0).astype(int)

df[delinq_cols + ["TotalDelinquency", "EverDelinquent", "Ever90DaysLate"]].describe()


,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate,TotalDelinquency,EverDelinquent,Ever90DaysLate
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,0.263233,0.082633,0.107920,0.453787,0.202420,0.055587
std,0.809436,0.534148,0.635481,1.664801,0.401805,0.229123
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,10.000000,10.000000,30.000000,1.000000,1.000000


## 2.1 Delinquency Feature Özeti

Özet delinquency değişkenlerine baktığımda tablo kabaca şöyle:

- Müşterilerin yaklaşık **%80’i hiç gecikme yaşamamış**, **%20’sinin** ise en az bir gecikmesi var.
- Ciddi gecikme (`Ever90DaysLate`) oranı **~%5.5**; bu, küçük ama oldukça riskli bir grup olduğunu gösteriyor.
- **TotalDelinquency**, üç gecikme tipinin toplamını veriyor ve capping sonrası 0–30 aralığında kalıyor; uç birkaç değer artık dağılımı bozmuyor.
- Uygulanan capping işlemiyle, delinquency kolonlarının hem ortalama/medyanı hem de max değerleri daha makul seviyelere inmiş durumda.

Bu haliyle delinquency tarafı, modele geçmiş ödeme davranışı hakkında güçlü ama kontrol edilebilir bir risk bilgisi taşıyor.

## 3. Limit ve Borç Kullanımı ile İlgili Feature’lar

Kredi kartı / kredi limiti kullanımı ve borç düzeyi, doğal olarak ilk bakılan risk göstergeleri. Bu adımda iki basit ama işlevli feature çıkarıyorum:

- **HighUtilizationFlag**: Limit kullanım oranı belirli bir eşik (ör. > 1.0) üzerindeyse 1, değilse 0.
- **DebtToIncomeRatio**: `DebtRatio` ile `MonthlyIncome` bilgisini bir araya getirip, borcun gelire oranını yaklaşık olarak temsil eden oran.

Bu iki değişken, özellikle **limiti sıkışık** ve **görece yüksek borç yükü** taşıyan müşterileri hızlıca görmeyi sağlıyor; ileride modelde de doğrudan kullanılacaklar.

In [4]:
# Yüksek limit kullanımı (örnek eşik: utilization > 1.0)
df["HighUtilizationFlag"] = (
    df["RevolvingUtilizationOfUnsecuredLines"] > 1.0
).astype(int)

# Basit borç / gelir oranı (sıfıra bölmeyi önlemek için +1)
df["DebtToIncomeRatio"] = df["DebtRatio"] / (df["MonthlyIncome"] + 1)

df[["RevolvingUtilizationOfUnsecuredLines", "HighUtilizationFlag", "DebtToIncomeRatio"]].head()


,RevolvingUtilizationOfUnsecuredLines,HighUtilizationFlag,DebtToIncomeRatio
0,0.766127,0,8.803663e-05
1,0.957151,0,4.685744e-05
2,0.658180,0,2.797022e-05
3,0.233810,0,1.092084e-05
4,0.907239,0,3.919812e-07


#### 3.1 Credit Utilization Feature Summary

Bu iki yeni değişken, müşterinin borç ve limit kullanım davranışını daha okunur hâle getiriyor:

- **HighUtilizationFlag**: Limiti tamamen kullanan ya da limitin üstüne çıkan müşterileri tek bir bayrakla ayırmamı sağlıyor.
- **DebtToIncomeRatio**: Borç yükünü gelire oranlayarak, "gelirine göre ne kadar borçlu?" sorusuna basit bir cevap veriyor.

İlk kontroller, her iki değişkenin de beklenen şekilde çalıştığını ve düşük riskli profillerde düşük, riskli profillerde daha yüksek değerler ürettiğini gösteriyor. Bu da modelleme aşamasında finansal stres/burçluluk etkisini daha net yakalamama yardımcı olacak.


### 4. Skewed Değişkenler için Log1p Dönüşümleri

EDA aşamasında aşağıdaki değişkenlerin dağılımı aşırı sağa çarpık çıkmıştı:

- `RevolvingUtilizationOfUnsecuredLines`
- `DebtRatio`
- `MonthlyIncome`

Bu tür kuyruklu dağılımlarda birkaç uç değer, histogramı neredeyse tamamen sıkıştırdığı için yorumlamayı zorlaştırıyor. Bu yüzden her bir değişken için `np.log1p(x)` (yani `log(1 + x)`) dönüşümünü uygulayıp `_log1p` son ekine sahip yeni sütunlar oluşturdum.

In [5]:
skewed_cols = [
    "RevolvingUtilizationOfUnsecuredLines",
    "DebtRatio",
    "MonthlyIncome",
]

for col in skewed_cols:
    new_col = f"{col}_log1p"
    df[new_col] = np.log1p(df[col])

df[[f"{c}_log1p" for c in skewed_cols]].describe()


,RevolvingUtilizationOfUnsecuredLines_log1p,DebtRatio_log1p,MonthlyIncome_log1p
count,150000.000000,150000.000000,150000.000000
mean,0.257900,1.525174,8.447405
std,0.384852,2.627237,1.195796
min,0.000000,0.000000,0.000000
25%,0.029430,0.161331,8.269757
50%,0.143391,0.312258,8.594339
75%,0.444074,0.625004,8.909370
max,10.833859,12.705832,14.917036


#### 4.1 Log1p Sonrası Dağılım Özeti

`RevolvingUtilizationOfUnsecuredLines_log1p`, `DebtRatio_log1p` ve `MonthlyIncome_log1p` için
özet istatistikler:

- Min, max ve çeyrek değerler daha makul aralıklara gelmiş durumda.
- Aşırı uç değerlerin etkisi azaldığı için dağılımlar daha kompakt görünüyor.
- Ana kitlenin hangi aralıklarda yoğunlaştığı, özet istatistiklerden artık daha net okunabiliyor.

Kısaca, log1p dönüşümü bu üç değişkeni hem görsel analiz hem de model eğitimi için daha “kullanılabilir” bir ölçeğe taşımış oldu.


# CHAPTER 2 : Risk Flags (Binary Risk Indicators)

Bu bölümde, EDA aşamasında öne çıkan kritik risk davranışlarını temsil eden binary (0/1) flag değişkenleri oluşturuyoruz. 

Bu tür değişkenler:

- kredi skorlama modellerinde güçlü sinyaller üretir,
- özellikle non-linear davranışları daha okunabilir hale getirir,
- delinquency geçmişi ve borç yükü gibi kavramları “var / yok” şeklinde sadeleştirir.

Oluşturulacak başlıca risk bayrakları:

- **HighDebtFlag** → Borç yükü gelire göre yüksekse  
- **HighUtilizationFlag** → Kredi kullanım oranı aşırı yüksekse  
- **EverDelinquent** → Müşteri geçmişte herhangi gecikme yaşamışsa  
- **Ever90DaysLate** → 90+ gün gecikme deneyimi varsa (yüksek risk göstergesi)  
- **MultipleDelinquencyFlag** → Birden fazla gecikme türünde geçmiş varsa  

Bu değişkenler, sonraki modelleme aşamasında güçlü ayrıştırıcı özellikler sağlayacak.



### 1. HighDebtFlag

Borç yükünün gelire göre aşırı yüksek olduğu durumları işaret eder.  
Banka tarafında Debt-to-Income Ratio (DTI) genellikle 0.4–0.5 aralığının üzerinde  olduğunda riskli kabul edilir. 
Bu yüzden `DebtToIncomeRatio > 0.4` olan müşterileri  `HighDebtFlag = 1` olarak işaretliyoruz.


In [6]:
# High Debt Flag (gelire göre aşırı borç yükü)
# Eşik banka içi politikalarda genellikle DTI > 0.4-0.5 olarak alınır.
df["HighDebtFlag"] = (df["DebtToIncomeRatio"] > 0.4).astype(int)

df["HighDebtFlag"].value_counts(normalize=True).round(3)


HighDebtFlag
0    0.928
1    0.072
Name: proportion, dtype: float64

HighDebtFlag dağılımı incelendiğinde, müşterilerin yaklaşık **%7.2’sinin**  gelirine kıyasla yüksek borç yüküne sahip olduğu görülüyor. 

Portföyün küçük ama riskli bu bölümü, kredi geri ödeme problemlerine daha yatkın olabileceği için model açısından kritik bir ayrıştırıcı rol oynuyor.

### 2. HighUtilizationFlag

Müşterinin revolving kredilerinde limit aşımı (kullanım oranı > 1) gibi durumları işaret eder. 
Bu flag, aşırı kredi kullanımını gösteren ve genellikle finansal stres altında  olmaya işaret eden binary bir değişkendir. 
Bu amaçla `RevolvingUtilizationOfUnsecuredLines > 1` olan müşterileri `HighUtilizationFlag = 1`olarak kodluyoruz.


In [7]:
df["HighUtilizationFlag"] = (df["RevolvingUtilizationOfUnsecuredLines"] > 1).astype(int)
df["HighUtilizationFlag"].value_counts(normalize=True).round(3)


HighUtilizationFlag
0    0.978
1    0.022
Name: proportion, dtype: float64

HighUtilizationFlag değişkeninin dağılımı, müşterilerin yalnızca **%2.2’sinin** kredi limitini aşan (utilization > 1) bir kullanım davranışı sergilediğini gösteriyor.

Bu küçük grup, finansal stres altında olma eğilimi nedeniyle default riski daha yüksek bir segmenti temsil ediyor. 
Dağılımın düşük olması, davranışın nadir ama riskli olduğunu düşündürüyor.

### 3. EverDelinquent & Ever90DaysLate

- **EverDelinquent:** Müşterinin geçmişte herhangi bir tipte gecikme yaşayıp yaşamadığını gösterir (0/1).  
- **Ever90DaysLate:** 90+ gün gecikme deneyimini işaret eder; bu, ağır risk sinyali olarak kabul edilir.

Bu iki flag, gecikme geçmişinin model için daha okunabilir hale getirilmesini sağlayan özet göstergelerdir.


In [8]:
delinq_cols = [
    "NumberOfTime30-59DaysPastDueNotWorse",
    "NumberOfTime60-89DaysPastDueNotWorse",
    "NumberOfTimes90DaysLate"
]

df["EverDelinquent"] = (df[delinq_cols].sum(axis=1) > 0).astype(int)
df["Ever90DaysLate"] = (df["NumberOfTimes90DaysLate"] > 0).astype(int)

df[["EverDelinquent", "Ever90DaysLate"]].mean()


EverDelinquent    0.202420
Ever90DaysLate    0.055587
dtype: float64

`EverDelinquent` için ortalama değer ~0.20. Yani müşterilerin kabaca **%20’si**  geçmişte en az bir kez gecikme yaşamış, **%80’i hiç gecikmemiş**. 
Dolayısıyla portföyde, gecikme deneyimi olan ama çoğunluk olmayan ayrı bir grup var.

`Ever90DaysLate` ortalaması ise ~0.055. 
Bu da müşterilerin sadece **%5–6’sının** 90+ gün gecikme yaşadığını gösteriyor. 
Sayı küçük ama bu grup “çok riskli köşe”yi temsil ediyor; 90+ gün gecikme, ödeme davranışında ciddi bozulma anlamına gelmekte ve 
modelin mutlaka yakalaması gereken bir sinyal.


### 4. MultipleDelinquencyFlag

Bir müşteride farklı gecikme türlerinin (30/60/90 gün) bir arada görülmesi, risk profilini daha ağır hale getirir. 

Bu nedenle, toplam gecikme sayısı 2'nin üzerinde olan müşterileri  `MultipleDelinquencyFlag = 1` olarak kodluyoruz. 
Böylece hem tekrarlı gecikme davranışı hem de farklı tür delinquency deneyimi, tek bir özet değişkende toplanmış oluyor.


In [9]:
df["MultipleDelinquencyFlag"] = (df[delinq_cols].sum(axis=1) >= 2).astype(int)

df["MultipleDelinquencyFlag"].value_counts(normalize=True).round(3)


MultipleDelinquencyFlag
0    0.913
1    0.087
Name: proportion, dtype: float64

MultipleDelinquencyFlag dağılımı, müşterilerin yaklaşık **%8.7’sinin** en az iki delinquency yaşadığını gösteriyor. 

Bu segment, hem tekrarlı gecikme davranışı hem de farklı tür gecikme deneyimi  bakımından daha ağır bir risk profilini temsil ediyor ve model için önemli 
bir uyarı sinyali görevi görüyor.


### 5. Delinquency Flag Doğrulama – Default Oran Karşılaştırması

Bu bölümde **sadece gecikme geçmişini temsil eden flag’ler** (`EverDelinquent`, `Ever90DaysLate`, `MultipleDelinquencyFlag`)
için default oranlarına bakıyoruz. Amaç, gecikme geçmişi arttıkça default oranının gerçekten yükselip yükselmediğini görmek.


In [10]:
flags = ["EverDelinquent", "Ever90DaysLate", "MultipleDelinquencyFlag"]

flat = (
    df.groupby(flags)["SeriousDlqin2yrs"]
      .mean()
      .reset_index()
      .sort_values(flags)
)

flat


,EverDelinquent,Ever90DaysLate,MultipleDelinquencyFlag,SeriousDlqin2yrs
0,0,0,0,0.027283
1,1,0,0,0.104352
2,1,0,1,0.243745
3,1,1,0,0.235849
4,1,1,1,0.486513


### 5.1 Risk Flag Doğrulama – Default Oranlarının İncelenmesi

Tabloyu kabaca şöyle okuyabiliriz:

- **(0, 0, 0)** satırı: Hiç gecikmesi olmayan müşteriler → default oranı ~**%2.7**
- **(1, 0, 0)** satırı: Sadece hafif gecikme (30–89 gün) yaşamış müşteriler → ~**%10.4**
- **(1, 0, 1)** satırı: 30/60 günlük gecikmesi olup, birden fazla tipte gecikme yaşayanlar → ~**%24.3**
- **(1, 1, 0)** satırı: En az bir delinquency ve 90+ gün gecikmesi olan ama “çoklu delinquency” eşiğini geçmeyenler → ~**%23.6**
- **(1, 1, 1)** satırı: Hem 90+ gün gecikmesi hem de birden fazla delinquency türü olan en riskli grup → ~**%48.7**

# CHAPTER 3 : Bining (Risk Segmentleri)

Bu bölümde sürekli değişkenleri daha yorumlanabilir risk segmentlerine ayırıyoruz.  

Amaç:

- Modelin non-lineer ilişkileri daha iyi yakalaması,
- SHAP ve iş tarafı raporlarında “segment bazlı” yorum yapabilmek (örn. 25–35 yaş arası, 3–6k gelir bandı vb.).

Bu aşamada aşağıdaki segmentler oluşturulacaktır:

- **AgeBin** → Yaş segmentleri  
- **IncomeBin** → Gelir segmentleri  
- **UtilizationBin** → Kredi kullanım oranı segmentleri  
- **DelinqBin** → Toplam gecikme segmentleri

### 1. AgeBin – Yaş Segmentleri

`AgeBin`, müşteri kitlesini dört temel yaş segmentine ayırır:

- **18–30**: Genç yetişkin segmenti (daha agresif kredi kullanımı)
- **31–45**: Orta yaş, gelir potansiyeli yüksek ana kitle
- **46–60**: Daha oturmuş finansal davranış, genelde daha dengeli profil
- **60+**: Emeklilik / daha düşük gelir riski barındıran segment

Bu segmentler hem modelde yaşın non-lineer etkisini yakalamak hem de “hangi yaş grubunda risk artıyor?” sorusuna net cevap verebilmek için
kullanılacaktır.


In [11]:
# Yaş segmentleri: 18–30 / 31–45 / 46–60 / 60+
import numpy as np

age_bins = [18, 30, 45, 60, np.inf]
age_labels = ["18-30", "31-45", "46-60", "60+"]

df["AgeBin"] = pd.cut(
    df["age"],
    bins=age_bins,
    labels=age_labels,
    right=True,
    include_lowest=True
)

df["AgeBin"].value_counts(dropna=False)


AgeBin
46-60    53636
60+      45060
31-45    40547
18-30    10757
Name: count, dtype: int64

In [12]:
df.groupby("AgeBin", observed=False)["SeriousDlqin2yrs"].mean().round(3)

AgeBin
18-30    0.116
31-45    0.093
46-60    0.068
60+      0.030
Name: SeriousDlqin2yrs, dtype: float64

#### Yorum – AgeBin

- Default oranı **yaş arttıkça kademeli olarak düşüyor**.
- **18–30** yaş grubu en riskli segment; yaklaşık **%11–12** default oranı var.
- **60+** segmenti ise en düşük riskte; default oranı **%3** civarında.

Özetle, yaş değişkeni kredi skorlama açısından anlamlı bir ayrıştırıcı; genç
müşterilerde temerrüt riski belirgin şekilde daha yüksek görünüyor.

### 2. IncomeBin – Gelir Segmentleri

`IncomeBin`, müşterileri aylık gelir seviyesine göre dört banda ayırır:

- **0–3k**: Daha düşük gelir; beklenmedik bir gider veya gelir kaybında ödemeleri aksatma riski daha yüksek grup  
- **3–6k**: Alt-orta gelir bandı – portföyün önemli bir kısmı
- **6–10k**: Orta-üst gelir – borç taşıma kapasitesi daha yüksek
- **10k+**: Yüksek gelir segmenti

Modelleme aşamasında her gelir bandının ayrı bir default oranı olacak; bu sayede özellikle düşük gelir bandındaki risk farkını daha net görebileceğiz.


In [13]:
# Gelir segmentleri: 0–3k / 3–6k / 6–10k / 10k+
income_bins = [0, 3000, 6000, 10000, np.inf]
income_labels = ["0-3k", "3-6k", "6-10k", "10k+"]

df["IncomeBin"] = pd.cut(
    df["MonthlyIncome"],
    bins=income_bins,
    labels=income_labels,
    right=True,
    include_lowest=True
)

df["IncomeBin"].value_counts(dropna=False)


IncomeBin
3-6k     73721
6-10k    32876
0-3k     25084
10k+     18319
Name: count, dtype: int64

In [14]:
df.groupby("IncomeBin", observed=False)["SeriousDlqin2yrs"].mean().round(3)

IncomeBin
0-3k     0.091
3-6k     0.069
6-10k    0.056
10k+     0.043
Name: SeriousDlqin2yrs, dtype: float64

#### Yorum – IncomeBin

- **Gelir azaldıkça default olma ihtimali artıyor.**
- **0–3k** gelir segmenti en riskli grup; default oranı yaklaşık **%9.1**.
- Gelir yükseldikçe risk kademeli olarak düşüyor; **10k+** grubunda oran **%4.3** seviyesine iniyor.

Bu sonuç, gelir değişkeninin müşterinin geri ödeme kapasitesini iyi yansıttığınıve model için güçlü bir ayrıştırıcı değişken olduğunu gösteriyor.


## 3. UtilizationBin – Kredi Kullanım Oranı

`UtilizationBin`, kredi kartı / açık kredi limitinin ne kadarının kullanıldığını gruplar:

- **0–30%**: Limitinin küçük bir kısmını kullanan, borç tarafı görece rahat müşteriler  
- **30–70%**: Günlük/normal kullanım seviyesi  
- **70–100%**: Limite yaklaşan, borç yükü artmaya başlayan müşteriler  
- **100%+**: Limiti tamamen dolu ya da aşmış, belirgin şekilde sıkışmış müşteriler

Bu gruplar, limit kullanımına göre riskin nasıl değiştiğini görmek için kullanılıyor.


In [15]:
# Kredi kullanım oranı segmentleri
# 0–30%  / 30–70%  / 70–100%  / 100%+
util_bins = [0, 0.3, 0.7, 1.0, np.inf]
util_labels = ["0-30%", "30-70%", "70-100%", "100%+"]

df["UtilizationBin"] = pd.cut(
    df["RevolvingUtilizationOfUnsecuredLines"],
    bins=util_bins,
    labels=util_labels,
    right=True,
    include_lowest=True
)

df["UtilizationBin"].value_counts(dropna=False)


UtilizationBin
0-30%      92882
30-70%     27170
70-100%    26627
100%+       3321
Name: count, dtype: int64

In [16]:
df.groupby("UtilizationBin", observed=False)["SeriousDlqin2yrs"].mean().round(3)

UtilizationBin
0-30%      0.022
30-70%     0.074
70-100%    0.177
100%+      0.372
Name: SeriousDlqin2yrs, dtype: float64

### Yorum – Kullanım Oranı ve Default

- Kredi kullanım oranı arttıkça default oranı doğrusal olmayan biçimde yükseliyor.  
- **0–30%** segmenti oldukça düşük riskte (~%2 civarı).  
- **70–100%** bandında oran ~%18’lere çıkıyor; limit neredeyse tamamen kullanılmış durumda.  
- **100%+** (limit aşımı) segmenti ise ~%37 civarında default oranı ile açık ara en riskli grup.

Sonuç olarak: Limit kullanım oranı, hem nakit sıkışıklığına hem de aşırı borçlanma eğilimine işaret ettiği için modelde güçlü bir uyarı değişkeni olarak kullanılabilir.

## 4. DelinqBin – Toplam Gecikme Sayısı Segmentleri

`DelinqBin`, müşteriyi toplam gecikme sayısına göre dört risk seviyesine ayırır:

- **0**: Hiç gecikme yaşamamış müşteri (en düşük risk)  
- **1**: Tekil gecikme – izlenmesi gereken ama henüz çok ağır değil  
- **2–3**: Birden fazla gecikme – davranışsal bozulma sinyali  
- **4+**: Süreklilik kazanan gecikme – en yüksek risk segmenti

Bu segment, hem `EverDelinquent` hem de `MultipleDelinquencyFlag` ile tutarlı şekilde gecikme yoğunluğunu sayısal olarak yakalar.


In [17]:
# Toplam gecikme sayısına göre segmentler
# 0 / 1 / 2–3 / 4+
delinq_bins = [-0.1, 0, 1, 3, np.inf]
delinq_labels = ["0", "1", "2-3", "4+"]

df["DelinqBin"] = pd.cut(
    df["TotalDelinquency"],
    bins=delinq_bins,
    labels=delinq_labels,
    right=True
)

df["DelinqBin"].value_counts(dropna=False)


DelinqBin
0      119637
1       17243
2-3      8826
4+       4294
Name: count, dtype: int64

In [18]:
df.groupby("DelinqBin", observed=False)["SeriousDlqin2yrs"].mean().round(3)

DelinqBin
0      0.027
1      0.122
2-3    0.276
4+     0.516
Name: SeriousDlqin2yrs, dtype: float64

### Yorum – Gecikme Sayısı ve Default

- Gecikme sayısı arttıkça default olasılığı çok hızlı yükseliyor.  
- Hiç gecikmesi olmayanlarda oran ~%2–3 civarında.  
- **1 gecikme** yaşayanlarda risk yaklaşık dört katına çıkıyor (~%12).  
- **2–3 gecikme** segmentinde oran ~%27 civarında.  
- **4+ gecikme** yaşayanlarda default oranı %50 seviyesini aşıyor.

Özet: Toplam gecikme sayısı, müşterinin finansal disiplinini doğrudan yansıttığı için,modelde en güçlü ayrıştırıcı değişkenlerden biri konumunda.


# CHAPTER 4 : Interaction Features

Bazı finansal göstergeler tek başına bakıldığında resmi tam anlatmıyor.  
Örneğin borç oranı yüksek ama kredi kullanım oranı düşükse risk daha sınırlı olabilir;  
aynı anda hem borç oranı hem de kullanım yüksek olduğunda ise tablo bambaşka görünür.

Bu yüzden, modelin bu tür birlikte ortaya çıkan davranışları görebilmesi için bazı temel interaction (çarpım) değişkenleri oluşturduk:

- **Utilization × DebtRatio** → Kredi kullanım oranı × borç oranı  
- **Income × Age** → Yaşa göre gelir düzeyi  
- **Delinquency × Utilization** → Gecikme geçmişi × mevcut limit kullanımı  
- **OpenLines × RealEstateLoans** → Açık kredi hatları × gayrimenkul kredi yükü  
- **HighUtilizationFlag × DebtRatio** → Yüksek kullanım flag’i × borç oranı

Bu etkileşimli değişkenler, özellikle XGBoost gibi ağaç tabanlı modellerde risk davranışını tek değişkenle yakalayamadığımız durumlarda ek sinyal taşıyacak.


In [19]:

# 1) Borç yükü × kredi kullanım oranı
df["Utilization_x_DebtRatio"] = df["RevolvingUtilizationOfUnsecuredLines"] * df["DebtRatio"]

# 2) Yaş × gelir (ekonomik kapasite göstergesi)
df["Income_x_Age"] = df["MonthlyIncome"] * df["age"]

# 3) Toplam gecikme × kredi kullanım oranı
df["Delinq_x_Utilization"] = df["TotalDelinquency"] * df["RevolvingUtilizationOfUnsecuredLines"]

# 4) Açık kredi hatları × gayrimenkul kredi hatları
df["OpenLines_x_RealEstate"] = (
    df["NumberOfOpenCreditLinesAndLoans"] * df["NumberRealEstateLoansOrLines"]
)

# 5) HighUtilizationFlag × DebtRatio (yüksek risk kombinasyonu)
df["HighUtil_x_DebtRatio"] = df["HighUtilizationFlag"] * df["DebtRatio"]

# İlk 5 gözlem
df[
    [
        "Utilization_x_DebtRatio",
        "Income_x_Age",
        "Delinq_x_Utilization",
        "OpenLines_x_RealEstate",
        "HighUtil_x_DebtRatio",
    ]
].head()


,Utilization_x_DebtRatio,Income_x_Age,Delinq_x_Utilization,OpenLines_x_RealEstate,HighUtil_x_DebtRatio
0,0.615186,410400.0,1.532253,78,0.0
1,0.116654,104000.0,0.000000,0,0.0
2,0.056020,115596.0,1.316360,0,0.0
3,0.008429,99000.0,0.000000,0,0.0
4,0.022614,3115812.0,0.907239,7,0.0


### Yorum

- **Utilization_x_DebtRatio**  
  - Değerler 0 ile 1 arasında küçük pozitif sayılar (ör: 0.61, 0.11, 0.05).  
  - Yüksek değer → hem borç oranı yüksek hem de limit kullanımı artmış.  
  - 0’a yakın değer → borç yükü / kullanım baskısı görece düşük.

- **Income_x_Age**  
  - Binler–milyonlar seviyesinde büyük çarpımlar üretiyor (99 000, 410 400, 3 115 812…).  
  - Amaç: modelin “yaşa göre gelir gücünü” hissetmesi.  
    Aynı gelir genç yaşta daha büyük, ileri yaşta daha küçük çarpım üretiyor.

- **Delinq_x_Utilization**  
  - Bazı satırlarda 0, bazılarında 1’in üzerinde değerler var (0.0, 1.53, 1.31, 0.90…).  
  - 0 civarı → ya hiç gecikme yok ya da limit kullanımı düşük.  
  - 1+ → hem gecikme geçmişi var hem de limit kullanımı yüksek; daha stresli profil.

- **OpenLines_x_RealEstate**  
  - İlk satırda 78 gibi uç bir değer, diğerlerinde 0 ve 7 gibi daha makul değerler.  
  - Çok sayıda açık hat + gayrimenkul kredisi olan müşterilerde çarpım büyüyor;  
    kredi ilişkisi yoğun ve karmaşık profilleri yukarı çekiyor.

- **HighUtil_x_DebtRatio**  
  - İlk 5 gözlemde hep 0.0.  
  - Sebep: bu değişken sadece `HighUtilizationFlag = 1` olduğunda pozitif oluyor.  
  - Yani dataset’te az görülen ama oluştuğunda “limit aşımı + yüksek borç oranı” kombinasyonunu işaret ediyor.

**Kısa özet:**  
Bu interaction feature’lar, tek tek değişkenlerle tam göremediğimiz “üst üste binen” risk durumlarını yakalamak için var. Küçük çarpımlar daha sıradan kredi profillerini, çok yüksek çarpımlar ise borç yükü/kullanım/kredi hatları tarafında baskısı yüksek müşterileri öne çıkarıyor.


# CHAPTER 5 : Domain Driven Features

Bu bölümde, sadece istatistiksel olarak anlamlı görünen değil, kredi risklendirme pratiğinde de anlamı olan özet değişkenler üretiyoruz.

Amaç, müşterinin borç yükü, gecikme şiddeti ve finansal stresini tek tek ham değişkenlerden okumak yerine, **daha doğrudan** gösteren skorlar oluşturmaktır.

Oluşturulan başlıca domain özellikleri:

- `EffectiveDebtLoad` → Müşterinin parasal borç yükü (borç oranı × gelir)
- `CreditLineDensity` → Yaşa göre kredi hattı yoğunluğu
- `RealEstateExposure` → Gayrimenkul kredilerinin toplam borca katkısı
- `DelinquencySeverityScore` → Gecikme sayısı ve şiddetini bir araya getiren skor
- `FinancialStressIndex` → Borç oranı ve kredi kullanımına bağlı genel stres göstergesi

Bu özellikler, modelin sadece “var mı/yok mu” değil, **borcun ağırlığı ve davranışın şiddeti**
üzerinden risk sinyali yakalamasına yardımcı olacaktır.



In [20]:

# 1) Gerçek borç yükü (parasal)
df["EffectiveDebtLoad"] = df["DebtRatio"] * df["MonthlyIncome"]

# 2) Yaşa göre kredi hattı yoğunluğu
df["CreditLineDensity"] = (
    df["NumberOfOpenCreditLinesAndLoans"] / df["age"]
)

# 3) Gayrimenkul borçlanma etkisi
df["RealEstateExposure"] = (
    df["NumberRealEstateLoansOrLines"] * df["DebtRatio"]
)

# 4) Ağırlıklı gecikme şiddeti skoru
df["DelinquencySeverityScore"] = (
    df["NumberOfTime30-59DaysPastDueNotWorse"] * 1
    + df["NumberOfTime60-89DaysPastDueNotWorse"] * 2
    + df["NumberOfTimes90DaysLate"] * 3
)

# 5) Finansal stres indeksi (borç oranı × kullanım)
df["FinancialStressIndex"] = np.log1p(
    df["DebtRatio"] * df["RevolvingUtilizationOfUnsecuredLines"]
)

# İlk birkaç gözlemi kontrol edelim
df[
    [
        "EffectiveDebtLoad",
        "CreditLineDensity",
        "RealEstateExposure",
        "DelinquencySeverityScore",
        "FinancialStressIndex",
    ]
].head()


,EffectiveDebtLoad,CreditLineDensity,RealEstateExposure,DelinquencySeverityScore,FinancialStressIndex
0,7323.197016,0.288889,4.817893,2,0.479450
1,316.878123,0.100000,0.000000,0,0.110337
2,258.914887,0.052632,0.000000,4,0.054507
3,118.963951,0.166667,0.000000,0,0.008393
4,1584.975094,0.142857,0.024926,1,0.022362


# Yorum 


- **EffectiveDebtLoad:** Parasal borç yükü müşteriden müşteriye ciddi farklılık gösteriyor. İlk müşteri yüksek borç baskısı altında (7323), diğerleri daha düşük seviyelerde.
- **CreditLineDensity:** Çoğu müşteri yaşına göre düşük kredi hattı yoğunluğuna sahip (0.05–0.28). Bu, daha az kredi deneyimi anlamına gelebilir.
- **RealEstateExposure:** Gayrimenkul kaynaklı borç yükü çoğunlukla sıfır; yalnızca bazı müşterilerde belirgin etki var.
- **DelinquencySeverityScore:** Gecikme şiddeti 0–4 arasında değişiyor. Skor 4 olan müşteriler ciddi gecikme geçmişine sahip.
- **FinancialStressIndex:** İlk müşteri yüksek stres seviyesinde (0.47), diğerleri düşük. Bu değişken borç oranı + kullanım kombinasyonundan oluşur.

**Sonuç : **  
Bu domain özellikleri müşterinin borç baskısı, geçmiş gecikmeleri ve genel finansal stresini tek bir bakışta özetleyen güçlü göstergelerdir.



# CHAPTER 6 : Feature Selection

Feature engineering sonrasında elimizde oldukça fazla sayıda değişken oluştu.  
Bu blokta amacımız:

- Birbirine **aşırı benzeyen (yüksek korelasyonlu)** feature’ları tespit etmek,
- **Multicollinearity** (değişkenler arası çoklu bağlantı) seviyesini ölçmek,
- Model için **gereksiz / zayıf** bazı değişkenleri aday olarak belirlemek,
- Sonraki blokta kullanılacak **final feature listesini** hazırlamaya zemin oluşturmak.

Bu aşamada sadece **teknik analiz** yapılacak; hangi kolonların gerçekten
atılacağına ilişkin kararları, elde ettiğimiz tablolara bakarak vereceğiz.


In [21]:
import numpy as np
import pandas as pd

TARGET = "SeriousDlqin2yrs"

# Yalnızca sayısal kolonları al (hedef dahil)
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# Hedef hariç feature listesi
feature_cols = [c for c in numeric_cols if c != TARGET]

len(feature_cols), feature_cols[:10]


(30,
 ['RevolvingUtilizationOfUnsecuredLines',
  'age',
  'NumberOfTime30-59DaysPastDueNotWorse',
  'DebtRatio',
  'MonthlyIncome',
  'NumberOfOpenCreditLinesAndLoans',
  'NumberOfTimes90DaysLate',
  'NumberRealEstateLoansOrLines',
  'NumberOfTime60-89DaysPastDueNotWorse',
  'NumberOfDependents'])

### 1. Hedef Değişken ile Korelasyon

İlk adımda, her bir sayısal değişkenin `SeriousDlqin2yrs` hedefiyle
olan korelasyonuna bakıyoruz. Bu:

- Hangi değişkenlerin hedefle **daha güçlü ilişki** taşıdığını,
- Hangilerinin **çok zayıf sinyal** verdiğini görmek için bir başlangıç noktasıdır.

Bu aşamada **hiçbir şey silmiyoruz**, sadece “güçlü / zayıf” adayları not alıyoruz.


In [22]:
# Sayısal kolonlar için korelasyon matrisi
corr_matrix = df[numeric_cols].corr()

# Hedef ile korelasyon (büyükten küçüğe sıralı)
target_corr = corr_matrix[TARGET].sort_values(ascending=False)

target_corr.to_frame()


,SeriousDlqin2yrs
SeriousDlqin2yrs,1.000000
MultipleDelinquencyFlag,0.357068
Ever90DaysLate,0.339577
TotalDelinquency,0.318768
DelinquencySeverityScore,0.314736
EverDelinquent,0.314407
NumberOfTimes90DaysLate,0.291353
NumberOfTime30-59DaysPastDueNotWorse,0.276157
NumberOfTime60-89DaysPastDueNotWorse,0.228412
HighUtilizationFlag,0.184146


### Yorum – Hedef Değişken ile Korelasyon

Tablo aslında üç ana grup gösteriyor:

**1. En güçlü sinyal: Gecikme değişkenleri**

`MultipleDelinquencyFlag`, `Ever90DaysLate`, `TotalDelinquency`,`DelinquencySeverityScore`, `EverDelinquent` ve 30–59 / 60–89 / 90+ gün gecikme sayıları hedefle **en yüksek korelasyona** sahip (≈ 0.22–0.36 aralığı).

- Yani model için en net sinyal, müşterinin **geçmiş gecikme davranışı**.
- Bu da FE tarafında ürettiğimiz delinquency flag’lerinin yerinde olduğunu gösteriyor:
  geçmişte gecikme yaşayanların default olma ihtimali belirgin şekilde artıyor.

**2. Orta seviye sinyal: Kullanım ve limit baskısı**

`HighUtilizationFlag` ve `RevolvingUtilizationOfUnsecuredLines_log1p` hedefle **orta seviyede** pozitif korelasyon taşıyor (~0.18 civarı).

- Limitini sürekli zorlayan, utilization’ı yüksek müşterilerde default oranının arttığına dair bir sinyal var.
- Bu yüzden hem `HighUtilizationFlag` hem de interaction / domain feature’lar 
   (örn. `HighUtil_x_DebtRatio`, `FinancialStressIndex`) modelde mantıklı adaylar.

**3. Zayıf / doğrusal olmayan ilişkiler**

Gelir (`MonthlyIncome`, `MonthlyIncome_log1p`), 
borç oranı (`DebtRatio`,`DebtRatio_log1p`, `DebtToIncomeRatio`), 
kredi hatları (`NumberOfOpenCreditLinesAndLoans`)
ve domain / interaction feature’ların çoğunun hedefle olan korelasyonu **sıfıra yakın**.

- Bu, “bu değişkenler işe yaramaz” demek değil; sadece **doğrusal tek yönlü
  ilişki zayıf** demek.
- Zaten bu yüzden bu değişkenleri:
- bin’lere böldük (`AgeBin`, `IncomeBin`, `UtilizationBin`, `DelinqBin`), interaction ve domain feature’lara çevirdik.
- Özellikle ağaç tabanlı modeller (Random Forest, XGBoost) bu tip zayıf korelasyonlu değişkenlerden bile **split’lerle** anlamlı sinyal çıkarabiliyor.

**4. Negatif ilişki: Yaş**

`age` hedefle hafif negatif korelasyona sahip (~ -0.12):

- Yaş arttıkça default olma ihtimali **biraz azalıyor**.
- Bu da segmentlediğimiz `AgeBin` sonuçlarıyla uyumlu: daha genç segmentlerde risk daha yüksek çıkmıştı.

**Özet**

- En güçlü sinyal **gecikme geçmişi + limit kullanımı** tarafında.
- Gelir / borç oranı gibi değişkenler tek başına çok yüksek korelasyon üretmiyor; bunları segmentler, interaction’lar ve domain skorları ile
  zenginleştirmek gerekiyor (bu FE bloklarında yaptığımız şey tam olarak bu).
- Bu aşamada tabloları sadece not ediyoruz; **hiçbir değişkeni sırf korelasyonu düşük diye silmiyoruz**. 
-Silme kararları, bir sonraki adımda multicollinearity + model performansı üzerinden verilecek.



### 2.Feature’lar Arası Yüksek Korelasyon

Sadece hedefle olan ilişki değil, **feature’ların birbirleriyle olan** ilişkisi de önemli:

- İki değişken birbirine **%90+ korele** ise, modelde aynı bilgiyi iki kez tutuyor olabiliriz.
- Bu durum hem **multicollinearity** yaratır, hem de modelin kararlılığını bozabilir.

Bu adımda **yüksek korelasyonlu feature çiftlerini** tablo olarak çıkarıyoruz.


In [23]:
# Sadece feature kolonları için korelasyon
feat_corr = df[feature_cols].corr()

high_corr_pairs = []
threshold = 0.9  

for i, col1 in enumerate(feature_cols):
    for j in range(i + 1, len(feature_cols)):
        col2 = feature_cols[j]
        corr_val = feat_corr.iloc[i, j]
        if abs(corr_val) >= threshold:
            high_corr_pairs.append((col1, col2, corr_val))

high_corr_df = pd.DataFrame(
    high_corr_pairs, columns=["feature_1", "feature_2", "corr"]
).sort_values("corr", ascending=False)

high_corr_df


,feature_1,feature_2,corr
0,DebtRatio,EffectiveDebtLoad,0.982442
1,MonthlyIncome,Income_x_Age,0.978513
3,TotalDelinquency,DelinquencySeverityScore,0.974149
2,NumberOfTimes90DaysLate,DelinquencySeverityScore,0.922088


### Feature’lar Arası Yüksek Korelasyon – Yorum

Bazı feature çiftleri birbirleriyle %90’dan yüksek korelasyon gösteriyor.  
Bu da iki değişkenin büyük ölçüde **aynı bilgiyi tekrar ettiği** ve modelde gereksiz yük oluşturduğu anlamına geliyor.

Öne çıkan eşleşmeler:

- **DebtRatio ↔ EffectiveDebtLoad**  
  EffectiveDebtLoad, zaten DebtRatio × MonthlyIncome formülünden geliyor; yani borç bilgisini daha “parasal” haliyle taşıyor.
- **MonthlyIncome ↔ Income_x_Age**  
  Income_x_Age, geliri yaş ile çarpan bir etkileşim; bu yüzden aylık gelirle neredeyse aynı hareket ediyor.
- **TotalDelinquency ↔ DelinquencySeverityScore**  
  DelinquencySeverityScore, toplam gecikmeleri ağırlıklandırarak özetleyen, daha rafine bir skor.
- **NumberOfTimes90DaysLate ↔ DelinquencySeverityScore**  
  Şiddet skorunda 90+ gün gecikmeye en yüksek ağırlık verildiği için bu ikisi de doğal olarak çok yakın.

Bu yüzden, **aynı bilgiyi iki kez taşımamak** için aşağıdaki tercihleri not ediyoruz:

**Öne çıkan / tutulması planlanan değişkenler:**

- `EffectiveDebtLoad`  → `DebtRatio` yerine parasal borç baskısını daha iyi özetliyor.
- `MonthlyIncome`      → Gelir bilgisinin yalın hali; etkileşim yerine ana değişkeni koruyoruz.
- `DelinquencySeverityScore` → Hem `TotalDelinquency` hem de `NumberOfTimes90DaysLate` bilgisini ağırlıklandırılmış tek bir skorla özetliyor.

**Gerekirse elenebilecek adaylar:**

- `DebtRatio`  
- `Income_x_Age`  
- `TotalDelinquency`  
- `NumberOfTimes90DaysLate`  

Bu aşamada henüz hiçbirini hemen silmiyoruz; sadece **final feature listesi**ni oluştururken hangi kolonların birbirinin yerine geçebileceğini netleştirmiş oluyoruz.

### 3.VIF Analizi (Multicollinearity Kontrolü)

Korelasyon bize ikili ilişkileri gösterir; fakat bir feature’ın **diğer tüm feature’larla** 
birlikte yarattığı bağlantıyı görmek için VIF (Variance Inflation Factor) kullanılır.

Genel pratik:

- VIF ≈ 1–5 → genelde kabul edilebilir,
- VIF > 5 → dikkat,
- VIF > 10 → ciddi multicollinearity riski, eleme adayı.

Bu adımda hangi feature’ların çok yüksek VIF’e sahip olduğunu listeleyeceğiz.


In [24]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df[feature_cols].copy()

vif_data = []
for i, col in enumerate(feature_cols):
    vif_val = variance_inflation_factor(X.values, i)
    vif_data.append((col, vif_val))

vif_df = pd.DataFrame(vif_data, columns=["feature", "VIF"]).sort_values("VIF", ascending=False)
vif_df


c:\Users\YAĞMUR\Masaüstü\credit-risk-model\.venv\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,feature,VIF
2,NumberOfTime30-59DaysPastDueNotWorse,inf
6,NumberOfTimes90DaysLate,inf
8,NumberOfTime60-89DaysPastDueNotWorse,inf
10,TotalDelinquency,inf
28,DelinquencySeverityScore,inf
3,DebtRatio,581.184947
25,EffectiveDebtLoad,564.205473
4,MonthlyIncome,40.622241
21,Income_x_Age,40.074644
5,NumberOfOpenCreditLinesAndLoans,39.420234


### 4. VIF Analizi Yorum

VIF değeri, bir değişkenin diğer tüm değişkenlerle ne kadar ilişkili olduğunu gösterir.  
Yüksek VIF, değişkenin modelde multicollinearity yarattığı anlamına gelir.

**Sonsuz (∞) VIF değerleri:**
- NumberOfTime30-59DaysPastDueNotWorse  
- NumberOfTimes90DaysLate  
- NumberOfTime60-89DaysPastDueNotWorse  
- TotalDelinquency  
- DelinquencySeverityScore  

Bu kolonlar birbirinin türevi olduğu için aralarında tam ilişki vardır.  
Bu gruptan sadece **DelinquencySeverityScore** tutulur, diğerleri çıkarılmalıdır.

**Çok yüksek VIF değerleri (100+):**
- DebtRatio  
- EffectiveDebtLoad  

Bu ikisi neredeyse tamamen aynı bilgiyi taşımaktadır.  
**Tutulması gereken:** EffectiveDebtLoad  
**Silinmesi gereken:** DebtRatio

**Yüksek VIF (20–50) grubunda izlenen yaklaşım:**
- **MonthlyIncome** → Temel gelir değişkeni, modelde tutulur.  
- **Income_x_Age** → MonthlyIncome ile çok yüksek ilişkili, çıkarılır.  
- **MonthlyIncome_log1p** → Aynı bilginin dönüşmüş hali, çıkarılır.  
- **CreditLineDensity** → Hem yüksek VIF hem de hedefle zayıf ilişki; elenmeye adaydır.  
- **NumberOfOpenCreditLinesAndLoans** → Yüksek VIF’e rağmen iş anlamı güçlü temel kredi hattı değişkeni, şimdilik tutulur.  
- **age** → Kritik demografik değişken, modelde tutulur.

**VIF < 10 olan değişkenler:**  
Bu gruptakiler multicollinearity açısından güvenlidir ve modelde kalabilir.

**Sonuç:**  
- VIF analizi, delinquency alt kolonlarının büyük kısmının atılması gerektiğini gösteriyor.  
- DebtRatio ve Income_x_Age gibi türev değişkenlerin elenmesi gerektiğini ortaya koyuyor.  
- Geri kalan değişkenlerin ise multicollinearity açısından rahatlıkla modelde tutulabileceği sonucuna varıyoruz.

### 5. Candidate Drop Listesi (Eleme Adayları)

Korelasyon analizi ve VIF sonuçlarına göre bazı değişkenlerin modelde tekrar eden bilgi taşıdığı veya çok yüksek multicollinearity 
oluşturduğu görülmüştür. Bu aşamada henüz kesin olarak silmiyoruz; ancak bu değişkenleri **“eleme adayı”** olarak işaretliyoruz.

Liste aşağıdaki kriterlere göre oluşturulmuştur:

- Hedef değişkenle zayıf ilişki (çok düşük korelasyon),
- Başka bir feature ile %90+ yüksek korelasyon (duplicate bilgi),
- VIF > 10 veya ∞ (multicollinearity riski),
- Aynı bilginin türevi / dönüşmüş hali (redundant feature),
- Domain gereği daha zayıf temsil (aynı kavramın daha iyi bir alternatifi varsa).

Bu liste, CHAPTER 7’de oluşturulacak **final feature set** için bir ara adımdır.
Model kurulumuna geçmeden önce hangi kolonların kesin olarak çıkarılacağı,
bu liste üzerinde yapılacak son değerlendirme ile belirlenecektir.



In [25]:
candidate_drop = [
    # Ham delinquency kolonları
    "NumberOfTime30-59DaysPastDueNotWorse",
    "NumberOfTimes90DaysLate",
    "NumberOfTime60-89DaysPastDueNotWorse",
    "TotalDelinquency",

    # Aşırı korelasyonlu / türev kolonlar
    "DebtRatio",
    "Income_x_Age",
    "MonthlyIncome_log1p",

    # Çok yüksek VIF + zayıf anlam
    "CreditLineDensity",
]


# CHAPTER 7 : Final Feature Export

Feature engineering (Blok 1–5) ve feature selection analizleri (Blok 6) sonrasında artık final modelleme aşamasında kullanılacak **nihai kolon listesini** çıkarmaya hazırız.

Bu blokta:

1. Feature selection sonucunda belirlenen `candidate_drop` listesinden  **silinmesi gereken kolonları** netleştireceğiz,
2. Kalan kolonlar ile **final eğitim veri setini** oluşturacağız,
3. Dosyayı `training_prepared.csv` olarak dışa aktaracağız.

Bu dosya, baseline model ve XGBoost modelleme adımlarının giriş verisi olacaktır.



In [26]:
from pathlib import Path
from src.config import DATA_DIR  # proje yapısında tanımlı

TRAIN_PREPARED_PATH = DATA_DIR / "training_prepared.csv"
TRAIN_PREPARED_PATH


WindowsPath('C:/Users/YAĞMUR/Masaüstü/credit-risk-model/data/training_prepared.csv')

### 1. Son Drop Listesinin Oluşturulması

`candidate_drop` listesi, Blok 6'da yapılan:

- korelasyon analizi,
- yüksek korelasyonlu feature çiftleri,
- VIF sonuçları,
- domain yorumları

göz önünde bulundurularak hazırlanmıştır.

Bu nedenle `candidate_drop`, final modele dahil edilmeyecek kolonları belirlemek
için temel referans olacaktır.

Bu hücrede bu listeyi **son haliyle** `final_drop` olarak tanımlıyoruz.


In [27]:
# Blok 6 analizlerine göre kesin olarak atılacak kolonlar
final_drop = [
    # Ham delinquency kolonları (yerine daha iyi temsilciler var)
    "NumberOfTime30-59DaysPastDueNotWorse",
    "NumberOfTimes90DaysLate",
    "NumberOfTime60-89DaysPastDueNotWorse",
    "TotalDelinquency",

    # Aşırı korelasyonlu / türev kolonlar
    "DebtRatio",
    "Income_x_Age",
    "MonthlyIncome_log1p",

    # Çok yüksek VIF + düşük anlamlılık
    "CreditLineDensity",
]

# Final kullanılacak kolonlar (hedef dahil)
final_cols = [c for c in df.columns if c not in final_drop]

len(final_cols), final_cols[:15]


(27,
 ['SeriousDlqin2yrs',
  'RevolvingUtilizationOfUnsecuredLines',
  'age',
  'MonthlyIncome',
  'NumberOfOpenCreditLinesAndLoans',
  'NumberRealEstateLoansOrLines',
  'NumberOfDependents',
  'EverDelinquent',
  'Ever90DaysLate',
  'HighUtilizationFlag',
  'DebtToIncomeRatio',
  'RevolvingUtilizationOfUnsecuredLines_log1p',
  'DebtRatio_log1p',
  'HighDebtFlag',
  'MultipleDelinquencyFlag'])

### 2. `training_prepared.csv` Dosyasının Oluşturulması

Bu noktaya kadar:

- eksik değerleri temizlenmiş,
- feature engineering adımları uygulanmış,
- feature selection analizleri ile son kolon listesi belirlenmiş

nihai eğitim veri setini hazırladık.

Bu blokta, bu **final kolon listesini** kullanarak tek bir DataFrame oluşturuyor ve  `data/training_prepared.csv` dosyasına kaydediyoruz.

Bu dosya, sonraki adımlarda:

- baseline modelleri,
- XGBoost modeli,
- hiperparametre optimizasyonu

için **standart giriş verisi (canonical training set)** olarak kullanılacaktır.


In [28]:
# Final dataframe oluştur
df_final = df[final_cols].copy()

# Dizin varsa oluştur
TRAIN_PREPARED_PATH.parent.mkdir(parents=True, exist_ok=True)

# Kaydet
df_final.to_csv(TRAIN_PREPARED_PATH, index=False)

print("Final eğitim datası kaydedildi:")
TRAIN_PREPARED_PATH


Final eğitim datası kaydedildi:


WindowsPath('C:/Users/YAĞMUR/Masaüstü/credit-risk-model/data/training_prepared.csv')